In [1]:
import sklearn.datasets, sklearn.cross_validation
import numpy as np
import sklearn.metrics
import copy

In [2]:
X = sklearn.datasets.load_boston().data 
y = sklearn.datasets.load_boston().target

In [3]:
X = np.array([np.array(list(x) + [1.]) for x in X])

In [4]:
BIG_CONSTANT = 10000

In [5]:
def predict(x_train, y_train, X_test, l=0.1):
    X = np.matrix(x_train)
    Y = np.matrix(y_train).T
    X_test_matrix = np.matrix(X_test)
    A = np.dot(X.T,X)
    A = A + l * np.eye(A.shape[0])
    A = A.getI()
    a = np.dot(A, np.dot(X.T, Y))
    #print np.array(np.dot(X_test_matrix, a))[:,0]
    return (np.dot(X_test_matrix, a).ravel()).tolist()

In [6]:
def add_algorithm_of_choosing_features(X, y, enough_improvment=1., number_folds=20, eye_addition = 0.01):
    kf = sklearn.cross_validation.KFold(len(X), n_folds=number_folds)
    number_features = len(X[0])
    current_feature_set = []
    total_loss = [0]
    current_improvment = BIG_CONSTANT
    while current_improvment > enough_improvment:
        remaining_features = list(set(range(number_features)).difference(set(current_feature_set)))
        remaining_features.sort()
        after_add_feature_loss = []

        for feature in remaining_features:
            feature_loss = []
            curr_features = current_feature_set + [feature]
            curr_features.sort()
            
            for train_index, test_index in kf:
                X_train, X_test = X[train_index], X[test_index]
                X_train, X_test = X_train[:, curr_features], X_test[:, curr_features]
                y_train, y_test = y[train_index], y[test_index]
                y_predict = predict(X_train, y_train, X_test, l = eye_addition)
                feature_loss.append(np.mean((y_predict - y_test) ** 2))

            after_add_feature_loss.append(np.mean(feature_loss))

        total_loss.append(min(after_add_feature_loss))
        current_feature_set.append(remaining_features[after_add_feature_loss.index(min(after_add_feature_loss))])
        current_feature_set.sort()
        current_improvment = total_loss[-1] - total_loss[-2]
    return current_feature_set, total_loss[-1]
#print loss, min(loss)

In [7]:
%%time
add_algorithm_of_choosing_features(X, y, enough_improvment=0.1)

CPU times: user 130 ms, sys: 0 ns, total: 130 ms
Wall time: 129 ms


([5, 12], 32.364173031756557)

In [8]:
def full_search_of_features(X, y,  number_folds=20, eye_addition = 0.01):
    kf = sklearn.cross_validation.KFold(len(X), n_folds=number_folds)
    number_features = len(X[0])
    print number_features
    min_loss = BIG_CONSTANT
    min_loss_set = []
    
    for k in range(1, 2 ** number_features):
        current_feature_set = []
        
        for feature in range(number_features):
            if (k >> feature) % 2 == 1:
                current_feature_set.append(feature)

        #print current_feature_set
        features_set_loss = []
        for train_index, test_index in kf:
            X_train, X_test = X[train_index], X[test_index]
            X_train, X_test = X_train[:, current_feature_set], X_test[:, current_feature_set]
            y_train, y_test = y[train_index], y[test_index]
            y_predict = predict(X_train, y_train, X_test, l = eye_addition)
            features_set_loss.append(np.mean((y_predict - y_test) ** 2))
                
        if np.mean(features_set_loss) < min_loss:
            min_loss = np.mean(features_set_loss)
            min_loss_set = current_feature_set
        
    return min_loss_set, min_loss

In [9]:
%%time
full_search_of_features(X, y)

14
CPU times: user 59.3 s, sys: 34.2 ms, total: 59.3 s
Wall time: 59.3 s


([0, 1, 2, 3, 5, 7, 11, 12], 29.044238594870087)

In [15]:
def del_algorithm_of_choosing_features(X, y, enough_improvment=1., number_folds=20, eye_addition = 0.01):
    kf = sklearn.cross_validation.KFold(len(X), n_folds=number_folds)
    number_features = len(X[0])
    current_feature_set = range(number_features)
    total_loss = [0]
    current_improvment = BIG_CONSTANT
   
    while current_improvment > enough_improvment:
        after_remove_feature_loss = []

        for feature in current_feature_set:
            curr_features = copy.copy(current_feature_set)
            curr_features.remove(feature)
            feature_loss = []
            
            for train_index, test_index in kf:
                X_train, X_test = X[train_index], X[test_index]
                X_train, X_test = X_train[:, curr_features], X_test[:, curr_features]
                y_train, y_test = y[train_index], y[test_index]
                y_predict = predict(X_train, y_train, X_test, l=eye_addition)
                feature_loss.append(np.mean((y_predict - y_test) ** 2))

            after_remove_feature_loss.append(np.mean(feature_loss))

        total_loss.append(min(after_remove_feature_loss))
        removing_feature = current_feature_set[after_remove_feature_loss.index(min(after_remove_feature_loss))]
        current_feature_set.remove(removing_feature)
        current_improvment = total_loss[-1] - total_loss[-2]
        
    return current_feature_set, total_loss[-1]    
    

In [16]:
%%time
del_algorithm_of_choosing_features(X,y)

CPU times: user 135 ms, sys: 0 ns, total: 135 ms
Wall time: 136 ms


([0, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13], 29.673960369091628)

## Вывод
На предложенной выборке алгоритм $del$ показал близкий в оптимальному результат. Алгоритм $add$ более далек от оптимального решения.